In [ ]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import time
import glob
import gc

import jax
import jax.numpy as jnp
from flax import nnx
from jax import grad, jit, vmap, random
import optax
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.abspath(os.path.join('../')))

In [ ]:
from geometric_bayesian.utils import DataLoader, plot_regression
rng_key = jax.random.key(0)
num_training_samples = 150
batch_size = 20
sigma_noise=0.3

# Split RNG key for reproducibility
rng_key, rng_noise = random.split(rng_key)

# Generate random training data
X_train = (
    random.uniform(rng_key, (num_training_samples, 1)) * 10
) 
noise = random.normal(rng_noise, X_train.shape) * sigma_noise
y_train = jnp.sin(X_train) + noise

# Generate testing data
X_test = jnp.linspace(-5, 15, 500).reshape(-1, 1)
y_test = jnp.sin(X_test)

# Create the training data loader
train_loader = DataLoader(X_train, y_train.squeeze(), batch_size)

In [ ]:
from geometric_bayesian.models import MLP
model = MLP(
    layers=[1,64,1],
)
num_params = model.size

In [ ]:
fig = plot_regression(model, X_test, y_test, X_train, y_train)

In [ ]:
n_epochs = 1000
step_size = 1e-2

optimizer = nnx.Optimizer(model, optax.adam(step_size))

def criterion(x, y):
    return jnp.mean((x - y) ** 2)

@nnx.jit
def train_step(model, optimizer, x, y):
    def loss_fn(model: MLP):
        y_pred = model(x)
        return criterion(y, y_pred)

    loss, grads = nnx.value_and_grad(loss_fn)(model)
    optimizer.update(grads) 

    return loss

In [ ]:
losses = []
for epoch in range(n_epochs):
    for x_tr, y_tr in train_loader:
        losses.append(train_step(model, optimizer, x_tr, y_tr))

    if epoch % 100 == 0:
        print(f"[epoch {epoch}]: loss: {losses[-1]:.4f}")

print(f'{optimizer.step.value = }')
print(f"Final loss: {losses[-1]:.4f}")

In [ ]:
fig = plot_regression(model, X_test, y_test, X_train, y_train)

In [ ]:
from geometric_bayesian.densities import Bernoulli, MultivariateNormal
from geometric_bayesian.functions.likelihood import neg_logll
from geometric_bayesian.operators import DiagOperator

p_ll = lambda f : Bernoulli(f, logits=True)
prior_var = DiagOperator(
    diag = jnp.array(10.), 
    dim = num_params
)
p_prior = MultivariateNormal(cov=prior_var)

In [ ]:
from geometric_bayesian.densities import MultivariateNormal
from geometric_bayesian.operators import DiagOperator

MultivariateNormal(model(X_train).squeeze(), DiagOperator(jnp.array(1.0), len(y_train)))._log(y_train)

In [ ]:
ggn_mv = ggn(
    model=model,
    train_data=(X_train, y_train),
    likelihood_density=Normal, 
    cov=jnp.array(1.0)
)

In [ ]:
graph_def, map_params = nnx.split(model)
def model_fn(input, params):
    return nnx.call((graph_def, params))(input)[0]

In [ ]:
num_params = sum(x.size for x in jax.tree.leaves(map_params))
eye_pytree = array_to_pytree(jnp.eye(num_params), map_params)
# precision = pytree_to_array(jax.lax.map(ggn_mv, eye_pytree, batch_size=None),axis=0)

In [ ]:
from geometric_bayesian.types import Vector, Matrix

def pf_jvp(input: Vector | Matrix, vector: Vector) -> Vector | Matrix:
    return jax.jvp(
        lambda p: model_fn(input=input, params=p),
        (map_params,),
        (vector,),
    )[1]

def pf_vjp(input: Vector | Matrix, vector: Vector | Matrix) -> Vector | Matrix:
    out, vjp_fun = jax.vjp(
        lambda p: model_fn(input=input, params=p), map_params
    )
    return vjp_fun(vector)

In [ ]:
grad_net = jax.lax.map(lambda p : pf_jvp(X_test[0], p), eye_pytree, batch_size=None).squeeze()

In [ ]:
pf_jvp(X_test[0], ggn_mv(pf_vjp(X_test[0],jnp.array([1.0]))[0]))

In [ ]:
tmp, _ = jax.tree.flatten(eye_pytree)

In [ ]:
tmp[0].shape

In [ ]:
pf_jvp(X_test[0],eye_pytree)